In [1]:
from IPython.core.debugger import set_trace

# importing required modules
import pandas as pd
import numpy as np
import time
import copy


In [2]:
def compact(placement_cpu, placement_job,num_job,num_mc):
    while True:
        move_count = 0
        for job_id,job in enumerate(placement_cpu):
            for i,task in enumerate(job):
                if i != num_job-1:

                    #CPU gap
                    curr_task_end = task[2] + task[3]
                    next_cpu_task_ini = job[i+1][2]
                    gap_cpu = next_cpu_task_ini - curr_task_end

                    #JOB gap
                    job_n = task[0]
                    task_n = task[1]
                    gap_job = 0
                    if task_n + 1 < num_job:
                        next_job_task_ini = placement_job[job_n][task_n+1][2]
                        gap_job = next_job_task_ini - curr_task_end
                    elif task_n + 1 == num_job:
                        gap_job = gap_cpu
                    #gap
                    gap = min(gap_cpu, gap_job)
                    #print(gap)

                    #Update init position: new_start = old_start + gap
                    if gap:
                        #Update placement_cpu array
                        task[2] = task[2] + gap  
                        #Update placement_job array
                        placement_job[job_n][task_n][2] = task[2]

                        move_count = move_count + 1

        if move_count == 0:
            break
    


In [3]:
def compute_gap(placement_cpu):
    gap = 0
    for job_id,job in enumerate(placement_cpu):
            start_point = job[0][2]
            stop_point = job[-1][2]+job[-1][3]
            span = stop_point - start_point
            duration = 0
            for i,task in enumerate(job):
                duration += task[3]
            gap += (span - duration)
    return gap

In [4]:
def metaheuristic_multiobjetive(num_job, num_mc, ms, pt, plot=False):


    num_gene=num_mc*num_job # number of genes in a chromosome
    
    population_size= 300 # default value is 30
    crossover_rate= 0.8 # default value is 0.8
    mutation_rate= 0.3 # default value is 0.2
    mutation_selection_rate=0.2
    num_iteration = 500 #2000
    
    num_mutation_jobs=round(num_gene*mutation_selection_rate)
    

    start_time = time.time()

    '''==================== main code ==============================='''
    '''----- generate initial population -----'''
    Tbest=999999999999999
    best_list,best_obj=[],[]
    population_list=[]
    makespan_record=[]
    for i in range(population_size):
        nxm_random_num=list(np.random.permutation(num_gene)) # generate a random permutation of 0 to num_job*num_mc-1
        population_list.append(nxm_random_num) # add to the population_list
        for j in range(num_gene):
            population_list[i][j]=population_list[i][j]%num_job # convert to job number format, every job appears m times

    for n in range(num_iteration):
        print(".",end = '')
        Tbest_now=99999999999           

        '''-------- two point crossover --------'''
        parent_list=copy.deepcopy(population_list)
        offspring_list=copy.deepcopy(population_list)
        S=list(np.random.permutation(population_size)) # generate a random sequence to select the parent chromosome to crossover

        for m in range(int(population_size/2)):
            crossover_prob=np.random.rand()
            if crossover_rate>=crossover_prob:
                parent_1= population_list[S[2*m]][:]
                parent_2= population_list[S[2*m+1]][:]
                child_1=parent_1[:]
                child_2=parent_2[:]
                cutpoint=list(np.random.choice(num_gene, 2, replace=False))
                cutpoint.sort()

                child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
                child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
                offspring_list[S[2*m]]=child_1[:]
                offspring_list[S[2*m+1]]=child_2[:]


        '''----------repairment-------------'''
        for m in range(population_size):
            job_count={}
            larger,less=[],[] # 'larger' record jobs appear in the chromosome more than m times, and 'less' records less than m times.
            for i in range(num_job):
                if i in offspring_list[m]:
                    count=offspring_list[m].count(i)
                    pos=offspring_list[m].index(i)
                    job_count[i]=[count,pos] # store the above two values to the job_count dictionary
                else:
                    count=0
                    job_count[i]=[count,0]
                if count>num_mc:
                    larger.append(i)
                elif count<num_mc:
                    less.append(i)

            for k in range(len(larger)):
                chg_job=larger[k]
                while job_count[chg_job][0]>num_mc:
                    for d in range(len(less)):
                        if job_count[less[d]][0]<num_mc:                    
                            offspring_list[m][job_count[chg_job][1]]=less[d]
                            job_count[chg_job][1]=offspring_list[m].index(chg_job)
                            job_count[chg_job][0]=job_count[chg_job][0]-1
                            job_count[less[d]][0]=job_count[less[d]][0]+1                    
                        if job_count[chg_job][0]==num_mc:
                            break     

        '''--------mutation--------'''   
        for m in range(len(offspring_list)):
            mutation_prob=np.random.rand()
            if mutation_rate >= mutation_prob:
                m_chg=list(np.random.choice(num_gene, num_mutation_jobs, replace=False)) # chooses the position to mutation
                t_value_last=offspring_list[m][m_chg[0]] # save the value which is on the first mutation position
                for i in range(num_mutation_jobs-1):
                    offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # displacement

                offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last # move the value of the first mutation position to the last mutation position


        '''--------fitness value(calculate makespan)-------------'''
        total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list) # parent and offspring chromosomes combination
        chrom_fitness,chrom_fit=[],[]
        total_fitness=0
        for m in range(population_size*2):
            j_keys=[j for j in range(num_job)]
            key_count={key:0 for key in j_keys}
            j_count={key:0 for key in j_keys}
            m_keys=[j+1 for j in range(num_mc)]
            m_count={key:0 for key in m_keys}
            placement_cpu = [[] for m in range(num_mc)] #NEW
            placement_job = [[] for m in range(num_job)] #NEW
            
            for i in total_chromosome[m]:
                gen_t=int(pt[i][key_count[i]])
                gen_m=int(ms[i][key_count[i]])
                j_count[i]=j_count[i]+gen_t
                m_count[gen_m]=m_count[gen_m]+gen_t

                if m_count[gen_m]<j_count[i]:
                    m_count[gen_m]=j_count[i]
                elif m_count[gen_m]>j_count[i]:
                    j_count[i]=m_count[gen_m]
                
                #array por cpu y task con [job,task,start,dur]
                placement_cpu[gen_m-1].append([i, key_count[i], j_count[i]-gen_t, gen_t]) 
                #array por job y task [cpu,task,start,dur]
                placement_job[i].append([gen_m-1, key_count[i], j_count[i]-gen_t, gen_t])
                
                key_count[i]=key_count[i]+1 
                
            # Compact jobs (ale)
            #print(placement_cpu)
            compact(placement_cpu, placement_job,num_job,num_mc)
            #print(placement_cpu)
            
            # Compute gaps (ale)
            gap = compute_gap(placement_cpu)
                        
            makespan=max(j_count.values()) + gap
            chrom_fitness.append(1/makespan)
            chrom_fit.append(makespan)
            total_fitness=total_fitness+chrom_fitness[m]

        '''----------selection(roulette wheel approach)----------'''
        pk,qk=[],[]

        for i in range(population_size*2):
            pk.append(chrom_fitness[i]/total_fitness)
        for i in range(population_size*2):
            cumulative=0
            for j in range(0,i+1):
                cumulative=cumulative+pk[j]
            qk.append(cumulative)

        selection_rand=[np.random.rand() for i in range(population_size)]

        for i in range(population_size):
            if selection_rand[i]<=qk[0]:
                population_list[i]=copy.deepcopy(total_chromosome[0])
            else:
                for j in range(0,population_size*2-1):
                    if selection_rand[i]>qk[j] and selection_rand[i]<=qk[j+1]:
                        population_list[i]=copy.deepcopy(total_chromosome[j+1])
                        break
                        
        '''----------comparison----------'''
        for i in range(population_size*2):
            if chrom_fit[i]<Tbest_now:
                Tbest_now=chrom_fit[i]
                sequence_now=copy.deepcopy(total_chromosome[i])
        if Tbest_now<=Tbest:
            Tbest=Tbest_now
            sequence_best=copy.deepcopy(sequence_now)

        makespan_record.append(Tbest)
        
        
    '''----------result----------'''
    print("optimal sequence",sequence_best)
    print("optimal value:%f"%Tbest)
    print('the elapsed time:%s'% (time.time() - start_time))
    
    
    # Compute placements
    
    m_keys=[j+1 for j in range(num_mc)]
    j_keys=[j for j in range(num_job)]
    key_count={key:0 for key in j_keys}
    j_count={key:0 for key in j_keys}
    m_count={key:0 for key in m_keys}


    placement_cpu = [[] for m in range(num_mc)]
    placement_job = [[] for m in range(num_job)] #NEW

    for i in sequence_best:
        gen_t=int(pt[i][key_count[i]])
        gen_m=int(ms[i][key_count[i]])
        j_count[i]=j_count[i]+gen_t
        m_count[gen_m]=m_count[gen_m]+gen_t

        if m_count[gen_m]<j_count[i]:
            m_count[gen_m]=j_count[i]
        elif m_count[gen_m]>j_count[i]:
            j_count[i]=m_count[gen_m]

        #array por cpu y task con [job,task,start,dur]
        placement_cpu[gen_m-1].append([i, key_count[i], j_count[i]-gen_t, gen_t]) 
        #array por job y task [cpu,task,start,dur]
        placement_job[i].append([gen_m-1, key_count[i], j_count[i]-gen_t, gen_t])

        key_count[i]=key_count[i]+1
    
    compact(placement_cpu, placement_job,num_job,num_mc)

    if plot:
        
        import matplotlib.pyplot as plt
        %matplotlib inline
        plt.plot([i for i in range(len(makespan_record))],makespan_record,'b')
        plt.ylabel('makespan',fontsize=15)
        plt.xlabel('generation',fontsize=15)
        plt.show()

    return Tbest, placement_cpu

In [5]:
if __name__ == "__main__":
    
    
    # Read problem instance
    filename = "datasets/inference/dataset_xsmall.data"

    with open(filename, "r") as file:
        NB_JOBS, NB_MACHINES = [int(v) for v in file.readline().split()]
        JOBS = [[int(v) for v in file.readline().split()] for i in range(NB_JOBS)]

    #-----------------------------------------------------------------------------
    # Prepare the data for modeling
    #-----------------------------------------------------------------------------

    # Build list of machines. MACHINES[j][s] = id of the machine for the operation s of the job j
    ms = [[JOBS[j][2 * s]+1 for s in range(NB_MACHINES)] for j in range(NB_JOBS)]


    # Build list of durations. DURATION[j][s] = duration of the operation s of the job j
    pt = [[JOBS[j][2 * s + 1] for s in range(NB_MACHINES)] for j in range(NB_JOBS)]


    Tbest,placements = metaheuristic_multiobjetive(NB_JOBS, NB_MACHINES, ms, pt, plot=True)
    

...........................

KeyboardInterrupt: 